# State class
틱택토 상태를 나타낼 수 있는 class를 만들어 주세요.

- 2명의 플레이어가 있고, 각 플레이어의 수가 뭔지 식별가능해야 합니다.  

**꼭 구현해야할 매소드(주니어/시니어 공통)**
- 둘 수 있는 위치가 어딘지 알려주는 함수
- win / lose / draw를 식별해주는 함수
- render 함수

**min-max algorithm / alpha-beta algorithm 구현(시니어)**

### 🚨 만약 감이 안 온다면 운영진에게 연락주시면 참고할 만한 레퍼런스를 보내드리겠습니다.
### 🚨 참고한 레퍼런스가 있다면, 출처를 남겨주세요.

[Reference]

https://ddddewang.tistory.com/entry/MINMAX-%EB%AF%B8%EB%8B%88%EB%A7%A5%EC%8A%A4-%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98-%EC%86%8C%EA%B0%9C

https://going-to-end.tistory.com/entry/%EC%95%8C%ED%8C%8C-%EB%B2%A0%ED%83%80-%EA%B0%80%EC%A7%80%EC%B9%98%EA%B8%B0-Alpha-beta-pruning


In [17]:
import numpy as np

In [19]:
class TicTacToe:
    def __init__(self):
        self.board = [[0 for _ in range(3)] for _ in range(3)]
        self.current_player = 1

    def reset(self):
        self.board = [[0 for _ in range(3)] for _ in range(3)]
        self.current_player = 1

    def render(self):
        players = {1: 'X', -1: 'O', 0: ' '}
        for i, row in enumerate(self.board):
            print(" " + " | ".join([players[cell] for cell in row]))
            if i < 2:
                print("---+---+---")

    def available_moves(self):
        return [(r, c) for r in range(3) for c in range(3) if self.board[r][c] == 0]

    def make_move(self, row, col):
        if self.board[row][col] == 0:
            self.board[row][col] = self.current_player
            self.current_player *= -1
            return True
        return False

    def check_winner(self):
        board = np.array(self.board)
        row_sum = np.sum(board, axis=1)
        col_sum = np.sum(board, axis=0)
        diag1_sum = np.trace(board)
        diag2_sum = np.trace(np.fliplr(board))
        all_sums = np.concatenate((row_sum, col_sum, [diag1_sum], [diag2_sum]))

        if 3 in all_sums:
            return 1
        if -3 in all_sums:
            return -1
        if not (board == 0).any():
            return 0
        return None

    def minimax(self, depth, is_maximizing):
        winner = self.check_winner()
        if winner is not None:
            return winner

        if is_maximizing:
            best_score = -float('inf')
            for (r, c) in self.available_moves():
                self.board[r][c] = 1
                score = self.minimax(depth + 1, False)
                self.board[r][c] = 0
                best_score = max(score, best_score)
            return best_score
        else:
            best_score = float('inf')
            for (r, c) in self.available_moves():
                self.board[r][c] = -1
                score = self.minimax(depth + 1, True)
                self.board[r][c] = 0
                best_score = min(score, best_score)
            return best_score

    def best_move_minimax(self):

        if self.current_player == -1:
            best_score = float('inf')
            move = None
            for (r, c) in self.available_moves():
                self.board[r][c] = -1
                score = self.minimax(0, True)
                self.board[r][c] = 0
                if score < best_score:
                    best_score = score
                    move = (r, c)
            return move
        else:
            best_score = -float('inf')
            for (r, c) in self.available_moves():
                self.board[r][c] = 1
                score = self.minimax(0, False)
                self.board[r][c] = 0
                if score > best_score:
                    best_score = score
                    move = (r, c)
            return move

    def alpha_beta(self, depth, alpha, beta, is_maximizing):
        winner = self.check_winner()
        if winner is not None:
            return winner

        if is_maximizing:
            max_eval = -float('inf')
            for (r, c) in self.available_moves():
                self.board[r][c] = 1
                eval = self.alpha_beta(depth + 1, alpha, beta, False)
                self.board[r][c] = 0
                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
            return max_eval
        else:
            min_eval = float('inf')
            for (r, c) in self.available_moves():
                self.board[r][c] = -1
                eval = self.alpha_beta(depth + 1, alpha, beta, True)
                self.board[r][c] = 0
                min_eval = min(min_eval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break
            return min_eval

    def best_move_alpha_beta(self):
        best_score = float('-inf') if self.current_player == 1 else float('inf')
        move = None

        for (r, c) in self.available_moves():
            self.board[r][c] = self.current_player

            if self.current_player == 1:
                score = self.alpha_beta(0, alpha=float('-inf'), beta=float('inf'), is_maximizing=False)
                if score > best_score:
                    best_score = score
                    move = (r, c)
            else:
                score = self.alpha_beta(0, alpha=float('-inf'), beta=float('inf'), is_maximizing=True)
                if score < best_score:
                    best_score = score
                    move = (r, c)

            self.board[r][c] = 0
        return move


In [20]:
def play_game():

    game = TicTacToe()
    game.reset()

    print("틱택토 게임을 시작합니다.")
    game.render()

    while True:

        if game.current_player == 1:
            print("플레이어 1 (X)의 차례입니다.")
            available = game.available_moves()
            print("가능한 위치: ", available)

            row, col = map(int, input("두고 싶은 위치(row, col)를 입력하세요: ").split())

            if (row, col) in available:
                game.make_move(row, col)
            else:
                print("잘못된 위치입니다. 다시 선택하세요.")
                continue
        else:
            print("플레이어 2 (O, AI)의 차례입니다.")
            move = game.best_move_alpha_beta() # 알고리즘 선택
            if move:
                print(f"AI가 선택한 위치: {move}")
                game.make_move(*move)

        game.render()

        result = game.check_winner()
        if result is not None:
            if result == 1:
                print("플레이어 1 (X)이 승리했습니다!")
            elif result == -1:
                print("플레이어 2 (O)이 승리했습니다!")
            else:
                print("무승부입니다.")
            break

play_game()


틱택토 게임을 시작합니다.
   |   |  
---+---+---
   |   |  
---+---+---
   |   |  
플레이어 1 (X)의 차례입니다.
가능한 위치:  [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
두고 싶은 위치(row, col)를 입력하세요: 0 1
   | X |  
---+---+---
   |   |  
---+---+---
   |   |  
플레이어 2 (O, AI)의 차례입니다.
AI가 선택한 위치: (0, 0)
 O | X |  
---+---+---
   |   |  
---+---+---
   |   |  
플레이어 1 (X)의 차례입니다.
가능한 위치:  [(0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
두고 싶은 위치(row, col)를 입력하세요: 1 1
 O | X |  
---+---+---
   | X |  
---+---+---
   |   |  
플레이어 2 (O, AI)의 차례입니다.
AI가 선택한 위치: (2, 1)
 O | X |  
---+---+---
   | X |  
---+---+---
   | O |  
플레이어 1 (X)의 차례입니다.
가능한 위치:  [(0, 2), (1, 0), (1, 2), (2, 0), (2, 2)]
두고 싶은 위치(row, col)를 입력하세요: 1 0
 O | X |  
---+---+---
 X | X |  
---+---+---
   | O |  
플레이어 2 (O, AI)의 차례입니다.
AI가 선택한 위치: (1, 2)
 O | X |  
---+---+---
 X | X | O
---+---+---
   | O |  
플레이어 1 (X)의 차례입니다.
가능한 위치:  [(0, 2), (2, 0), (2, 2)]
두고 싶은 위치(row, col)를 입력하세요: 2 2
 O | X |  
---+---+---
 X | 